In [34]:
import sys
sys.path.append(r"D:\8th Semester\Machine learning\ML Labs")

In [35]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [36]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [37]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [38]:
model1 = create_rnn()
model1.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_8 (SimpleRNN)             │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_9 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [41]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th Semester\Machine learning\ML Labs\lab9'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [42]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [43]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [44]:
path_dataset =r'D:\8th Semester\Machine learning\ML Labs\DATASET'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [45]:
time_steps=24
num_features=21

In [46]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.5021905899047852 sec


In [47]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/5
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0951 - mae: 0.0951 - mape: 5270.5317  
Epoch 1: val_loss improved from inf to 0.01881, saving model to D:\8th Semester\Machine learning\ML Labs\lab9\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 0.0950 - mae: 0.0950 - mape: 5262.5547 - val_loss: 0.0188 - val_mae: 0.0188 - val_mape: 7.9456
Epoch 2/5
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0158 - mae: 0.0158 - mape: 20.9449
Epoch 2: val_loss improved from 0.01881 to 0.01202, saving model to D:\8th Semester\Machine learning\ML Labs\lab9\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0158 - mae: 0.0158 - mape: 20.9997 - val_loss: 0.0120 - val_mae: 0.0120 - val_mape: 4.9906
Epoch 3/5
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0132 - mae: 0.0132 - mape: 7.5705 
Epoch 3: val_loss did not improve from 0.01202
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0132 - mae: 0.0132 - mape: 7.6908 - val_loss: 0.0137 - val_mae: 0.0137 - val_mape: 6.7626
Epoch 4/5
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0121 - mae: 0.0121 - mape: 132.2904 
Epoch 4: val_loss improved from 0.01202 to 0.01097, saving model to D:\8th Semester\Machine learning\ML Labs\lab9\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0121 - mae: 0.0121 - mape: 132.2774 - val_loss: 0.0110 - val_mae: 0.0110 - val_mape: 4.9643
Epoch 5/5
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0114 - mae: 0.0114 - mape: 32.5647  
Epoch 5: val_loss improved from 0.01097 to 0.00954, saving model to D:\8th Semester\Machine learning\ML Labs\lab9\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0114 - mae: 0.0114 - mape: 32.5329 - val_loss: 0.0095 - val_mae: 0.0095 - val_mape: 4.1032


In [50]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
from keras.losses import MeanAbsoluteError as MAELoss  # For recompiling if needed

# Define model path
model_path = r'D:\8th Semester\Machine learning\ML Labs\lab9\E1-cp-0005-loss0.01.h5'

# Load the model with custom_objects mapping 'mae' to actual class
model = load_model(
    model_path,
    compile=False,  # Don't try to compile with legacy config
    custom_objects={
        'mae': MeanAbsoluteError,  # Fix: map 'mae' string to real object
        'mean_absolute_error': MeanAbsoluteError,
        'mean_absolute_percentage_error': MeanAbsolutePercentageError
    }
)

# OPTIONAL: Re-compile the model (required if you'll train it or need metrics)
model.compile(
    loss=MAELoss(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
)

# Prediction
y_pred_scaled = model.predict(test_X)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Evaluation Metrics
MAE = np.mean(np.abs(y_pred - y_test_unscaled))
MEDAE = np.median(np.abs(y_pred - y_test_unscaled))
MSE = np.mean(np.square(y_pred - y_test_unscaled))
RMSE = np.sqrt(MSE)
MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100

# Print results
print('Mean Absolute Error (MAE):', round(MAE, 2))
print('Median Absolute Error (MedAE):', round(MEDAE, 2))
print('Mean Squared Error (MSE):', round(MSE, 2))
print('Root Mean Squared Error (RMSE):', round(RMSE, 2))
print('Mean Absolute Percentage Error (MAPE):', round(MAPE, 2), '%')
print('Median Absolute Percentage Error (MDAPE):', round(MDAPE, 2), '%')
print('\nShape of y_test_unscaled:', y_test_unscaled.shape)
print('Shape of y_pred:', y_pred.shape)


379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step
Mean Absolute Error (MAE): 144.44
Median Absolute Error (MedAE): 114.36
Mean Squared Error (MSE): 35704.0
Root Mean Squared Error (RMSE): 188.96
Mean Absolute Percentage Error (MAPE): 0.99 %
Median Absolute Percentage Error (MDAPE): 0.8 %

Shape of y_test_unscaled: (12105, 1)
Shape of y_pred: (12105, 1)


In [51]:
checkpoints = r'D:\8th Semester\Machine learning\ML Labs\lab9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th Semester\Machine learning\ML Labs\lab9\E1-cp-0004-loss0.01.h5'
start_epoch= 7

In [55]:
import os
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError

# Define model path and checkpoint path
model_path = r"D:\8th Semester\Machine learning\ML Labs\lab9\E1-cp-0005-loss0.01.h5"
checkpoints = r"D:\8th Semester\Machine learning\ML Labs\lab9\best_model.h5"

# Construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(
    checkpoints,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Set callbacks
callbacks = [EpochCheckpoint1]

# If no model exists, build a new one
if model_path is None or not os.path.exists(model_path):
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

# Otherwise, load model from disk
else:
    print(f"[INFO] loading model from {model_path}...")

    model = load_model(
        model_path,
        compile=False,
        custom_objects={
            'mae': MeanAbsoluteError,
            'mean_absolute_error': MeanAbsoluteError,
            'mape': MeanAbsolutePercentageError,
            'mean_absolute_percentage_error': MeanAbsolutePercentageError
        }
    )

    # Re-compile with fresh optimizer
    opt = Adam(learning_rate=1e-4)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

    print("[INFO] model recompiled with new learning rate:", model.optimizer.learning_rate.numpy())


[INFO] loading model from D:\8th Semester\Machine learning\ML Labs\lab9\E1-cp-0005-loss0.01.h5...
[INFO] model recompiled with new learning rate: 1e-04


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [19]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 197.4838
Epoch 00001: val_loss improved from inf to 0.01016, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 113s 43ms/step - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 197.4838 - val_loss: 0.0102 - val_mean_absolute_error: 0.0102 - val_mean_absolute_percentage_error: 4.7394
Epoch 2/5
2652/2653 [============================>.] - ETA: 0s - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 248.9462
Epoch 00002: val_loss improved from 0.01016 to 0.01001, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 130s 49ms/step - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 248

In [58]:
import numpy as np
from keras.models import load_model
from keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
from sklearn.preprocessing import MinMaxScaler  # only if scaler was MinMaxScaler
# Ensure you have test_X, test_y, and scaler already defined above this

# Load the model with proper custom_objects
model = load_model(
    r'D:\8th Semester\Machine learning\ML Labs\lab9\E1-cp-0005-loss0.01.h5',
    custom_objects={
        'mae': MeanAbsoluteError(),
        'mape': MeanAbsolutePercentageError()
    }
)

# Predict on test data
y_pred_scaled = model.predict(test_X)

# Inverse scale predictions and true values
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Calculate error metrics
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE):', np.round(MAE, 2))

MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE):', np.round(MEDAE, 2))

MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE):', np.round(MSE, 2))

RMSE = np.sqrt(MSE)
print('Root Mean Squared Error (RMSE):', np.round(RMSE, 2))

MAPE = np.mean(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Mean Absolute Percentage Error (MAPE):', np.round(MAPE, 2), '%')

MDAPE = np.median(np.abs((y_test_unscaled - y_pred) / y_test_unscaled)) * 100
print('Median Absolute Percentage Error (MDAPE):', np.round(MDAPE, 2), '%')

print('\n\nShapes:')
print('y_test_unscaled.shape:', y_test_unscaled.shape)
print('y_pred.shape:', y_pred.shape)


379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Mean Absolute Error (MAE): 144.44
Median Absolute Error (MedAE): 114.36
Mean Squared Error (MSE): 35704.0
Root Mean Squared Error (RMSE): 188.96
Mean Absolute Percentage Error (MAPE): 0.99 %
Median Absolute Percentage Error (MDAPE): 0.8 %


Shapes:
y_test_unscaled.shape: (12105, 1)
y_pred.shape: (12105, 1)
